In [138]:
from astropy.nddata import CCDData
from astropy.stats import mad_std

import ccdproc as ccdp
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem
from astropy.io import fits

from pathlib import Path


In [139]:
def calculating_stats(path, image_type):
    file_collection = ccdp.ImageFileCollection(path)
    files = file_collection.summary['imagetyp'] == image_type
    times = set(file_collection.summary['exptime'][files])
    if times:
        for exp_time in sorted(times):
            calibrated_files = file_collection.files_filtered(imagetyp=image_type, exptime=exp_time, include_path=True)
    else:
        raise(TypeError(f'Image type {image_type} does not exist'))
    pixel_values = np.empty(shape=(len(calibrated_files), 16389120))
    for index, file in enumerate(calibrated_files):
        with fits.open(file) as hdul:
            data = hdul[0].data.flatten()
            pixel_values = np.insert(pixel_values, index, data, axis = 0)
    pixel_values = pixel_values[:len(calibrated_files)]
    means = np.mean(pixel_values, axis = 0)
    std = np.std(pixel_values, axis = 0)
    standard_error_mean = sem(pixel_values, axis = 0)
    return means, std, standard_error_mean

In [141]:
dark_files_path = Path('2022_03_20', 'Dark')
dark_imagetype = 'Dark Frame'

dark_mean, dark_std, dark_sem = calculating_stats(dark_files_path, dark_imagetype)

In [142]:
bias_path = Path('2022_03_20', 'Bias')
bias_imagetyp = 'Bias Frame'

bias_mean, bias_std, bias_sem = calculating_stats(bias_path, bias_imagetyp)

In [143]:
flat_path = Path('2022_03_20', 'Flat', 'Luminance')
flat_imagetyp = 'Flat Frame'


flat_mean, flat_std, flat_sem = calculating_stats(flat_path, flat_imagetyp)